# Datos climáticos diarios

Se generan las series climáticas diarias de las estaciones principales de la AEMET. Se pueden descargar todas las disponibles o por provincias. La duración de la serie también es definible

In [1]:
import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import pandas as pd
import numpy as np
import datetime
import time

In [10]:
# Ruta en la que está guardada la API key
ruta_API = "F:/Series/AEMET/apikey_AEMET.txt"
# Ruta donde se guardarán las series
#ruta_series = 'F:/Series/AEMET/Clima/Diario/'
ruta_series = 'F:/Series/AEMET/OpenData/Diario/Esla/'

# Definir el inicio y fin de la serie, y la duración de cada periodo
start_date = datetime.datetime(1950, 1, 1, 0, 0, 0)
final_date = datetime.datetime(2017, 11, 30, 23, 59, 59)

# Definir las estaciones: 'todas' o la lista de con los códigos
estaciones = ['2620E', '2621', '2622', '2624']

## 1. Estaciones

In [3]:
def dms2dd(degrees, minutes, seconds, direction):
    """It converts a latitude or longitude value given in degrees, minutes, seconds and direction to decimal degrees
    """
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(3600);
    if direction == 'W' or direction == 'S':
        dd *= -1
    return dd

In [4]:
# Carga la api key 
api_key = open(ruta_API).read().rstrip()
querystring = {"api_key": api_key}

# Obtenemos información de todas las estaciones disponibles
url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones"

In [5]:
# Realizamos la petición
r = requests.get(url, params=querystring, verify=False)

In [6]:
# Obtenemos el link del que descargaremos los datos
data_url = r.json()['datos']
r_data = requests.get(data_url, params=querystring, verify=False)

In [7]:
# Vemos el contenido
stations = r_data.json()

In [8]:
# Crea un data frame con las estaciones meteorológicas y sus características
n = len(stations)

columnas = []
for key in stations[0].keys():
    columnas.append(key)

stn_AEMET = pd.DataFrame(index=np.arange(n), columns=columnas)
stn_AEMET = stn_AEMET.loc[:, ['indicativo', 'nombre', 'provincia', 'indsinop',
                              'latitud', 'longitud', 'altitud']]

i = 0
for stn in stations:
    stn_AEMET.indicativo[i] = stn['indicativo']
    stn_AEMET.nombre[i] = stn['nombre']
    stn_AEMET.provincia[i] = stn['provincia']
    stn_AEMET.indsinop[i] = stn['indsinop']
    stn_AEMET.latitud[i] = dms2dd(stn['latitud'][:2], stn['latitud'][2:4], stn['latitud'][4:6], stn['latitud'][6])
    stn_AEMET.longitud[i] = dms2dd(stn['longitud'][:2], stn['longitud'][2:4], stn['longitud'][4:6], stn['longitud'][6])
    stn_AEMET.altitud[i] = int(stn['altitud'])
    i += 1
stn_AEMET.sort_values('indicativo', inplace=True)
stn_AEMET.set_index('indicativo', drop=True, inplace=True)

print('Nº de estaciones disponibles: ', stn_AEMET.shape[0])

stn_AEMET.head(2)

Nº de estaciones disponibles:  291


,nombre,provincia,indsinop,latitud,longitud,altitud
indicativo,,,,,,
0002I,VANDELLÒS,TARRAGONA,08169,40.9581,0.871389,32
0016A,REUS AEROPUERTO,TARRAGONA,08175,41.145,1.16361,71


In [11]:
stn_AEMET.to_csv(ruta_series + 'estaciones_AEMET.csv', sep=',', na_rep='-100', header=True, index=True)

## 2. Extraer datos diarios

### 2.1 Funciones

In [12]:
def parse_data(raw_data):
    """Convert from string to floating point number those columns in the data that represent quantitative variables"""
    data = []
    for d in raw_data:
        d = dict(d)  # Exto copia el parámetro
        for param in ['prec', 'presMax', 'presMin', 'racha', 'sol', 'tmax', 'tmed', 'tmin', 'velmedia', 'altitud', 'dir']:
            try:
                d[param] = float(d[param].replace(',', '.'))
            except:
                d[param] = None
        data.append(d)
    
    return data

In [13]:
def generate_dates(start_date, final_date, step):
    """It generates a list of dates beginning in the 'start_date', ending in the 'final_date' and every 'step' number of 
    dates"""
    
    chunks = [start_date]

    next_date = start_date + step
    while next_date < final_date:
        chunks.append(next_date)
        next_date += step

    chunks.append(final_date)

    return chunks

In [14]:
def extract_data(station, periodos):

    raw_data = []

    for ii in range(1, len(periodos)):
        print()
        print(station,
              " - ",
              periodos[ii-1].strftime('%Y-%m-%dT%H:%M:%SUTC'),
              " - ",
              periodos[ii].strftime('%Y-%m-%dT%H:%M:%SUTC'))

        url = ("https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/"
               "fechaini/{start}/fechafin/{end}/estacion/{station}".format(
                   start=periodos[ii-1].strftime('%Y-%m-%dT%H:%M:%SUTC'),
                    end=periodos[ii].strftime('%Y-%m-%dT%H:%M:%SUTC'),
                    station=station)
              )

        iterate = True

        while iterate:

            r = requests.get(url, params=querystring, verify=False)

            # Si no me deja hacer la conexión, la repito  
            iterate = (r.status_code == requests.codes.too_many_requests)

            print(r.json())

            # Chequeo si la petición ha ido bien    
            if r.status_code == requests.codes.ok:
                # Hago la petición para obtener los datos
                data_url = r.json()['datos']
                r_data = requests.get(data_url, params=querystring, verify=False)

                # INCONSISTENCIA DE LA API:
                # Cuando no encuentra datos en el rango seleccionado, la API devuelve
                # que el status code es 200 (todo ok) y devuelve un json con el error
                # cuando encuentra, no hay atributo estado            
                try:
                    estado = r_data.json()['estado']
                except:
                    estado = 200


                # Si ha ido bien guardo los datos
                if estado == requests.codes.ok:
                    #print(r_data.json())
                    raw_data.extend(r_data.json())
                else:
                    print(r_data.json()['descripcion'])
            else:
                print(r.json()['descripcion'])

            time.sleep(60/45)
            
    return raw_data

In [15]:
def serie_df(data):
    """Create a data frame with the series of daily records of a station given as a list of dictionaries"""
    # Crea un data frame con los datos diarios de una estación
    n = len(data)

    columnas = []
    for key in data[0].keys():
        columnas.append(key)

    serie = pd.DataFrame(index=np.arange(n), columns=columnas)
    serie = serie.loc[:, ['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'prec', 'tmed', 'tmin', 'horatmin', 'tmax', 
                          'horatmax', 'dir', 'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin',
                          'horaPresMin']]

    for i in np.arange(len(data)):
        keys = list(data[i].keys())
        for key in keys:
            serie[key][i] = data[i][key]
    
    serie.fecha = pd.to_datetime(serie.fecha, yearfirst=True)
    
    return serie

### 2.2 Generar series

#### Definir fechas

In [16]:
# Definir la duración de cada periodo y calcular los periodos
step = datetime.timedelta(days=30)
periodos = generate_dates(start_date, final_date, step)
print('Nº de periodos en los que se divide el periodo de estudio: ', len(periodos))

Nº de periodos en los que se divide el periodo de estudio:  828


#### Definir las estaciones

In [17]:
# Definir las estaciones a extraer
if estaciones[0] == 'todas':
    stn_AEMET_sub = stn_AEMET
else:
    stn_AEMET_sub = stn_AEMET.loc[estaciones]
        
print('Nº de estaciones a extraer: ', stn_AEMET_sub.shape[0])

KeyError: "None of [['2620E', '2621', '2622', '2624']] are in the [index]"

#### Extraer, organizar y exportar las series

In [ ]:
# Extraer, reorganizar y exportar las series de cada una de las estaciones
for stn in stn_AEMET_sub.index[3:]:
    # Cargar la API key
    api_key = open(ruta_API).read().rstrip()
    querystring = {"api_key": api_key}
    
    # Extraer los datos
    raw_data = extract_data(stn, periodos)
    
    # Convertir los datos en numeros reales
    data = parse_data(raw_data)

    # Reorganizar la serie en forma de data frame
    serie = serie_df(data)
    
    # Exportar la serie
    serie.to_csv(ruta_series + stn + '_' + stn_AEMET_sub.loc[stn, 'nombre'] + '.csv', index=None, na_rep='NaN')


5530E  -  1950-01-01T00:00:00UTC  -  1950-01-31T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/3a214d9b', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1950-01-31T00:00:00UTC  -  1950-03-02T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/00ea56e0', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1950-03-02T00:00:00UTC  -  1950-04-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/7c19776b', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1950-04-01T00:00:00UTC  -  1950-05-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/318f9fa9', 'metadatos': 'https://opendata.aemet.es/op

No hay datos que satisfagan esos criterios

5530E  -  1952-07-19T00:00:00UTC  -  1952-08-18T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/331087cf', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1952-08-18T00:00:00UTC  -  1952-09-17T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/ed1746df', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1952-09-17T00:00:00UTC  -  1952-10-17T00:00:00UTC
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límit

No hay datos que satisfagan esos criterios

5530E  -  1954-01-10T00:00:00UTC  -  1954-02-09T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/c80ea6db', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1954-02-09T00:00:00UTC  -  1954-03-11T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/3a2914d0', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1954-03-11T00:00:00UTC  -  1954-04-10T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/98e45f1a', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1954-04-10T00:00:00UTC  -  1954-05-10T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/340734ed',

No hay datos que satisfagan esos criterios

5530E  -  1955-07-04T00:00:00UTC  -  1955-08-03T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/dc71d113', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1955-08-03T00:00:00UTC  -  1955-09-02T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/63f3b881', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1955-09-02T00:00:00UTC  -  1955-10-02T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/7a8d530f', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1955-10-02T00:00:00UTC  -  1955-11-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/654b1014',

No hay datos que satisfagan esos criterios

5530E  -  1956-12-25T00:00:00UTC  -  1957-01-24T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/eaa05495', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1957-01-24T00:00:00UTC  -  1957-02-23T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/f272c7a8', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1957-02-23T00:00:00UTC  -  1957-03-25T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/2c67cacb', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1957-03-25T00:00:00UTC  -  1957-04-24T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/431716ef',

...


5530E  -  1965-07-11T00:00:00UTC  -  1965-08-10T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/c09d89eb', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1965-08-10T00:00:00UTC  -  1965-09-09T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/91342b62', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1965-09-09T00:00:00UTC  -  1965-10-09T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/0f5a93ac', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1965-10-09T00:00:00UTC  -  1965-11-08T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/4b62f984', 'metadatos': 'https://opendata.aemet.es/op

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/3642eb06', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1967-03-03T00:00:00UTC  -  1967-04-02T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/bc8797b0', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1967-04-02T00:00:00UTC  -  1967-05-02T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/5a25650e', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criterios

5530E  -  1967-05-02T00:00:00UTC  -  1967-06-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/ba0b504d', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}
No hay datos que satisfagan esos criteri

{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Es

No hay datos que satisfagan esos criterios

5530E  -  1971-03-12T00:00:00UTC  -  1971-04-11T00:00:00UTC
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
{'descripcion': 'Límite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.', 'estado': 429}
Límite de peticiones o caudal por minuto excedido para este usuario. Espere al si


5530E  -  1972-10-02T00:00:00UTC  -  1972-11-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/0ed2e75f', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

5530E  -  1972-11-01T00:00:00UTC  -  1972-12-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/90beb185', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

5530E  -  1972-12-01T00:00:00UTC  -  1972-12-31T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e63e0a10', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

5530E  -  1972-12-31T00:00:00UTC  -  1973-01-30T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e9fa4289', 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

5530E  -  1973-01-30T00:00:00UTC  -  1973-03-01T00:00:00UTC
{'descripcion': 'exito', 'estado': 200, 'datos'